# Twitter Sentiment Analysis

`Author:` Iqra Saher

## DataSet Information
Our task is to classify the Positive and Negative Sentiments by Using Naive Bayes Algorithm.

For training the model, we have a labelled dataset containing `13871 rows and 21 columns`. But we are interested only `text and sentiment` columns on dataset for performing Sentiment Analyis. On sentiment Column we have `Postisive, Negative and Neutral` sentiments But we will do Binary Classification so only consider (Postivie,Negative) Sentiments and then we will apply Naive Bayes Agorithm.


### import Libraries

In [ ]:
!pip install nltk

In [ ]:
nltk.download('all')

In [26]:
import pandas as pd
import numpy as np 
import nltk
#for stemming and lemmatization
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

### Reading the Dataset

In [27]:
df = pd.read_csv('Sentiment.csv')
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [28]:
df = df[['text','sentiment']]
df

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       13871 non-null  object
 1   sentiment  13871 non-null  object
dtypes: object(2)
memory usage: 216.9+ KB


In [30]:
df = df[df['sentiment']!='Neutral']
df

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


### Data Preprocessing

In [31]:
# We will spilt the data for training and testing
train,test =train_test_split(df,test_size=0.1)

In [32]:
train

,text,sentiment
5655,RT @rj4gui4r: Is anyone keeping track of how m...,Negative
6174,I can't wait for the #SNL impersonation of Don...,Negative
12710,"RT @RWSurferGirl: Thanks Fox News, you're rais...",Positive
11571,RT @RWSurferGirl: After @GovChristie hugged Ob...,Negative
7326,So were some of the biggest positive audience ...,Negative
...,...,...
4227,How is it Hollywood can't get Dr. Doom right o...,Positive
10031,Huckabee is such a card. #Gopdebates #VoteYall,Negative
8422,RT @RWSurferGirl: So @megynkelly posed for adu...,Negative
12201,#PlannedParenthood should be funded in Florida...,Negative


#### As we are interested in Only tweet text so we remove all @,#tags,stopwords,etc. So we will create a regex that will match of all the text structure and we will get our desire text.

In [33]:
#define pattern
pattern = "(#\w+)|(RT\s@\w+:)|(rt\s@\w+:)(http.*)|(@\w+)"

In [34]:
ps = PorterStemmer()
lemmatizer =WordNetLemmatizer()

In [35]:
def clean_text(data):
    tweets = []
    sentiments = []
    for index, row in data.iterrows():
        sentence = re.sub(pattern,'',row.text)
        words = [e.lower() for e in sentence.split()]
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
        words = ' '.join(words)
        tweets.append(words)
        sentiments.append(row.sentiment)
    return tweets,sentiments

In [36]:
train_tweets,train_sentiments =clean_text(train)

In [ ]:
train_tweets

In [38]:
final_data = {'tweets':train_tweets,'sentiments':train_sentiments}

In [39]:
processed_data = pd.DataFrame(final_data)

In [40]:
processed_data

,tweets,sentiments
0,anyone keeping track many time lincoln roll gr...,Negative
1,can't wait impersonation donald trump,Negative
2,"thanks fox news, raising 's ratings. 🇺🇸",Positive
3,hugged obama deserve stage tonight. 🇺🇸,Negative
4,biggest positive audience response rhetoric ac...,Negative
...,...,...
9651,hollywood can't get dr. doom right foxnews got...,Positive
9652,huckabee card.,Negative
9653,posed adult pictures.should bring up?,Negative
9654,funded florida - floridians...,Negative


In [41]:
labelencoder = LabelEncoder()
processed_data['sentiments'] = labelencoder.fit_transform(processed_data['sentiments'])

In [42]:
processed_data

,tweets,sentiments
0,anyone keeping track many time lincoln roll gr...,0
1,can't wait impersonation donald trump,0
2,"thanks fox news, raising 's ratings. 🇺🇸",1
3,hugged obama deserve stage tonight. 🇺🇸,0
4,biggest positive audience response rhetoric ac...,0
...,...,...
9651,hollywood can't get dr. doom right foxnews got...,1
9652,huckabee card.,0
9653,posed adult pictures.should bring up?,0
9654,funded florida - floridians...,0


### Converting Words into Vectors

In [43]:
cv = CountVectorizer(ngram_range=(1,3))
cv.fit(processed_data['tweets'])

CountVectorizer(ngram_range=(1, 3))

In [44]:
x_train =cv.transform(processed_data['tweets'])

In [45]:
print(x_train.shape)

(9656, 91164)


In [46]:
#x_train.toarray()

In [47]:
target = processed_data['sentiments'].values
target

array([0, 0, 1, ..., 0, 0, 0])

### Sentiment Analysis (Model Building)

In [48]:
classifier = MultinomialNB()

In [49]:
classifier.fit(x_train.toarray(),target)

MultinomialNB()

In [50]:
test_tweets , test_sentiments = clean_text(test)

In [51]:
test_tweets

['"the purpose military kill people break things"-mike huckabee',
 "16% u.s. home tv set watched last night's - 3x highest-rated debate 2012 election http://t.co/ele3lmeoxo",
 'woman start protesting misogyny "stay vagina unless want there!" placards. (i …',
 'jeb bush reminds elevator music. hear listen. 🇺🇸',
 'long realises bad party? narrower narrower becomes.',
 'honey badger candidate round. great tonight!',
 'posed adult pictures.should bring up?',
 'candidate attack fox stoping speaking. 🇺🇸 …',
 'pointed out, huge audience poc ready learn. learned mostly scary. best, tone deaf.',
 'turned msnbc opinion . talking trump trump trump. huh? listening night?',
 'issue va veterans?',
 'fact moderator attacking candidate show debate large group interview.',
 "cheered! shame, : trump responds megyn kelly's question misogyny – misogyny http://t.c…",
 'early favourite compared tax tithing. keep religion dude!',
 'fox moderator throw major shade second-tier candidates. http://t.co/xrblfem4f

In [52]:
data_test = {'tweets':test_tweets,'sentiments':test_sentiments}

In [53]:
final_test_data =pd.DataFrame(data_test)

In [54]:
final_test_data

,tweets,sentiments
0,"""the purpose military kill people break things...",Negative
1,16% u.s. home tv set watched last night's - 3x...,Positive
2,"woman start protesting misogyny ""stay vagina u...",Negative
3,jeb bush reminds elevator music. hear listen. 🇺🇸,Negative
4,long realises bad party? narrower narrower bec...,Negative
...,...,...
1068,cancel primaries. fox party set &amp; anointed...,Negative
1069,caught last night…and lost appetite watching d...,Negative
1070,zingers. go away.,Negative
1071,re: problem!ucan continue 2ignore fact evidenc...,Negative


In [55]:
x_test = cv.transform(final_test_data['tweets'])

In [56]:
x_test

<1073x91164 sparse matrix of type '<class 'numpy.int64'>'
	with 16445 stored elements in Compressed Sparse Row format>

In [57]:
x_test.shape

(1073, 91164)

In [58]:
y_pred = classifier.predict(x_test.toarray())

In [60]:
final_test_data['sentiments'] = labelencoder.fit_transform(final_test_data['sentiments'])

In [62]:
final_test_data

,tweets,sentiments
0,"""the purpose military kill people break things...",0
1,16% u.s. home tv set watched last night's - 3x...,1
2,"woman start protesting misogyny ""stay vagina u...",0
3,jeb bush reminds elevator music. hear listen. 🇺🇸,0
4,long realises bad party? narrower narrower bec...,0
...,...,...
1068,cancel primaries. fox party set &amp; anointed...,0
1069,caught last night…and lost appetite watching d...,0
1070,zingers. go away.,0
1071,re: problem!ucan continue 2ignore fact evidenc...,0


In [65]:
actual_values = final_test_data['sentiments'].values
actual_values

array([0, 1, 0, ..., 0, 0, 1])

In [67]:
print(accuracy_score(y_pred,actual_values))

0.8443616029822927


### Output

1. After removing `Neutral` Sentiments, we have `10729` tweets available in our dataset available for performing Binary Classificatio on `Positive ,Negative` Sentiments.
2. After spliting `10%` for training, we have `9656` tweets available for testing.
3. After performing LabelEncoding ,it changes
    * 0 : Negative Sentiment
    * 1 : Positive Sentiment
4. We Apply Naive Bayes Algorithm and achieve `84%` accuracy.